### LIVRO: PESQUISA OPERACIONAL PARA OS CURSOS DE ADMINISTRAÇÃO E ENGENHARIA, 4ª EDIÇÃO.
#### AUTORES: ERMES MEDEIROS DA SILVA, ELIO MEDEIROS DA SILVA, VALTER GONÇALVES E AFRÂNIO CARLOS MUROLO.




**OBSERVAÇÃO:**

A BASE DO EXEMPLO DE SIMULAÇÃO ABAIXO FOI RETIRADA DO LIVRO CITADO ACIMA, PORÉM TIVE QUE ADAPTAR OS DADOS, POIS NÃO FOI POSSÍVEL SABER AO CERTO OS DADOS DE ENTRADA (DEMANDA SEMANAL). PORTANTO, NESTE NOTEBOOK MOSTRO A IDEIA DO EXEMPLO DO LIVRO COM VALORES DIFERENTES DE ENTRADA.

In [1]:
# DESATIVAR MENSAGENS DE WARNINGS (warn = -1) E ATIVAR (warn = 0)
options(warn = -1)
suppressPackageStartupMessages({library(dplyr)})

#### SIMULAÇÃO DE MONTE CARLO
#### EXEMPLO DE APLICAÇÃO

UM FEIRANTE FAZ COMPRA DE OVOS UMA VEZ POR SEMANA NUM ENTREPOSTO ATACADISTA. OS OVOS NÃO VENDIDOS DENTRO DE UMA SEMANA SE ESTRAGAM, E SÃO DESCARTADOS, ACARRETANDO PREJUÍZO DE 400 u.m POR DÚZIA. POR OUTRO LADO, A FALTA DE PRODUTO PARA A VENDA TAMBÉM ACARRETA PERDA, ESTIMADA EM 150 u.m POR DÚZIA DEMANDADA E NÃO VENDIDA. O FEIRANTE ANOTOU A DEMANDA DAS ÚLTIMAS 40 SEMANAS E DIVIDIU-AS EM SETE CLASSES, CONFORME ABAIXO:

#### DADOS

In [2]:
# DADOS DA DEMANDA SEMANAL DE DÚZIAS
dados_semanais <- c(210,216,222,216,210,228,234,240,234,216,222,216,204,210,222,228,240,234,216,228,
                    252,248,252,256,252,248,234,240,234,248,216,228,222,234,228,222,234,240,234,228)

#### TESTAR AS HIPÓTESES

1. COMPRAR CADA SEMANA A DEMANDA EFETIVA DA SEMANA ANTERIOR.
2. COMPRAR UMA QUANTIDADE IGUAL À MÉDIA HISTÓRICA ANOTADA NO PERÍODO ANTERIOR DE 40 SEMANAS (MÉDIA = VALOR INTEIRO MAIS PRÓXIMO DA MÉDIA VERIFICADA).
3. O EXAME DOS RESULTADOS SUGERE O TESTE DE OUTRA HIPÓTESE.

In [3]:
summary(dados_semanais)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  204.0   220.5   228.0   229.9   240.0   256.0 

In [4]:
n <- length(dados_semanais); cat("Tamanho(n):",n)
R <- max(dados_semanais) - min(dados_semanais); cat("\nAplitude(R):",R)
k <- round(sqrt(n),2); cat("\nClasses(k):",k)
h <- round(R/k, 2); cat("\nIntervalo de classe(h):",h)

Tamanho(n): 40
Aplitude(R): 52
Classes(k): 6.32
Intervalo de classe(h): 8.23

#### TABELA DE FREQUÊNCIA

In [5]:
# INTERVALOS
intervalos <- seq(204, 260, 6); cat("Intervalos:", intervalos)
# CLASSES
classes <- cut(x = dados_semanais, breaks = intervalos, right = FALSE, include.lowest = TRUE)
# FREQUÊNCIA 
frequencia <- table(classes)
frequencia_df <- data.frame(frequencia)
# TABELA DE FREQUÊNCIA
Tabela.Frequencia <- frequencia_df %>%
mutate(Freq.R.1000 = Freq/n * 1000,
       Freq.R.A = cumsum(Freq)/n * 1000)

Intervalos: 204 210 216 222 228 234 240 246 252 258

In [6]:
# CRIANDO UM VETOR COM O PONTO MÉDIO DAS CLASSES DA TABELA DE FREQUÊNCIA
PM = c()
num = 0; soma = 1
for(x in 1:length(frequencia_df$classes)){
    num = num + soma
    ponto.medio <- (intervalos[num] + intervalos[num+1])/2
    PM <- append(PM, ponto.medio)
}     

In [7]:
# ADICIONANDO O VETOR À TABELA DE FREQUÊNCIAS
Tabela.Frequencia <- cbind(Tabela.Frequencia, PM)
# COLOCANDO AS COLUNAS DA TABELA NA ORDEM DESEJADA
Tabela.Frequencia <- select(Tabela.Frequencia, classes,PM,everything())

VAMOS SIMULAR A PRIMEIRA HIPÓTESE PARA 20 SEMANAS. OS LIMITES PARA OS NÚMEROS ALEATÓRIOS SÃO OBTIDOS ATRAVÉS DA FREQUÊNCIA ACUMULADA RELATIVA, CONFORME MOSTRA A TABELA.

In [8]:
# CRIANDO LIMITES INFERIORES E SUPERIORES PARA SORTEIO
Tabela.Frequencia <- Tabela.Frequencia %>%
mutate(L.I.sorteio = Freq.R.A - Freq.R.1000,
       L.S.sorteio = Freq.R.A - 1); Tabela.Frequencia

classes,PM,Freq,Freq.R.1000,Freq.R.A,L.I.sorteio,L.S.sorteio
<fct>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
"[204,210)",207,1,25,25,0,24
"[210,216)",213,3,75,100,25,99
"[216,222)",219,6,150,250,100,249
"[222,228)",225,5,125,375,250,374
"[228,234)",231,6,150,525,375,524
"[234,240)",237,8,200,725,525,724
"[240,246)",243,4,100,825,725,824
"[246,252)",249,3,75,900,825,899
"[252,258]",255,4,100,1000,900,999


In [9]:
# FUNÇÕES ESPECÍFICAS PARA ESTE PROBLEMA DE NEGÓCIO
# CRIANDO A FUNÇÃO VENDA - (DEMANDA E ESTOQUE INICIAL)
funcao_venda = function(x,y){
    if(x < y)
        x 
    else
        y 
}
# CRIANDO A FUNÇÃO DE ESTOQUE FINAL - (ESTOQUE INICIAL - VENDA)
funcao_estoque.final = function(x,y){
        z = x - y
}
# CRIANDO A FUNÇÃO CUSTO DE FALTA - (DEMANDA - ESTOQUE INICIAL)*150.00
funcao_custo.falta = function(x,y){
    if(x > y)
        x = (x - y) * 150.00
    else
        y = 0
}
# CRIANDO FUNÇÃO CUSTO DE SOBRA - (ESTOQUE FINAL*400)
funcao_custo.sobra = function(x){
        y = x * 400
}
# CRIANDO A FUNÇÃO CUSTO TOTAL - (CUSTO FALTA + CUSTO SOBRA)
funcao_custo.total = function(x, y){
        z = x + y
}

#### FUNÇÃO SIMULAÇÃO DE MONTE CARLO

In [10]:
# FUNÇÃO SIMULAÇÃO DE MONTE CARLO
SimulacaoMonteCarlo <- function(iteracoes,coluna_L.I,coluna_L.S,coluna_PM){
    Simulacao <- data.frame()
    for (i in 1:iteracoes){
        iteracao <- i
        sorteio <- sample(0:999, 1)
        contador = 0
        repeat {
        contador = contador + 1
        if (sorteio >= coluna_L.I[contador] & sorteio <= coluna_L.S[contador]){
        ponto.medio.classe <- coluna_PM[contador]
        vetor = c(iteracao,sorteio,ponto.medio.classe)
        Simulacao = rbind(Simulacao, vetor)
        colnames(Simulacao) <- c("Iteracoes","Sorteio","Previsao")
        break()
        
    }
    } 
}
    return(Simulacao)
}

#### TESTANDO A PRIMEIRA HIPÓTESE
1. COMPRAR CADA SEMANA A DEMANDA EFETIVA DA SEMANA ANTERIOR.

#### CRIANDO A TABELA DE CUSTOS

A PARTIR DA TABELA DE FREQUÊNCIA COM OS NÚMEROS ALEATÓRIOS, LEVANTAMOS UMA SEQUÊNCIA DE VINTE NÚMEROS DE TRÊS ALGARISMOS, COMO JÁ ANTERIORMENTE DESCRITO. PARA INICIAR O PROCESSO, VAMOS ACEITAR A DEMANDA ANTERIOR À PRIMEIRA SEMANA IGUAL À MÉDIA PONDERADA DO PERÍODO ANTERIOR.

In [11]:
# COMPRAR CADA SEMANA A DEMANDA EFETIVA DA SEMANA ANTERIOR - ESTOQUE INICIAL
# MÉDIA PONDERADA DOS DADOS FOI COLOCADA NO ESTOQUE INICIAL DA PRIMEIRA SEMANA, POIS NÃO SE TEM A DEMANDA ANTERIOR, JÁ QUE 
# A SIMULAÇÃO COMEÇA NA PRIMEIRA ITERAÇÃO.
media.ponderada <- round(weighted.mean(Tabela.Frequencia$PM, Tabela.Frequencia$Freq))
Estoque.Inicial <- c(media.ponderada); Estoque.Inicial

[1] 233

In [12]:
Tabela.Custos <- SimulacaoMonteCarlo(20, Tabela.Frequencia$L.I.sorteio, Tabela.Frequencia$L.S.sorteio, Tabela.Frequencia$PM)
colnames(Tabela.Custos) <- c("Semana", "Sorteio", "Demanda")

In [13]:
# CRIANDO A COLUNA ESTOQUE INICIAL 
# A PARTIR DAS PREVISÕES DE DEMANDA DA PRIMEIRA SEMANA COLOCA-SE NO ESTOQUE INICIAL A DEMANDA DA SEMANA ANTERIOR.
Tabela.Custos$Estoque.Inicial[1] <- Estoque.Inicial
Tabela.Custos$Estoque.Inicial[2:20] <- Tabela.Custos$Demanda[1:19]

In [14]:
# CRIANDO A COLUNA VENDA
Tabela.Custos$Venda <- mapply(funcao_venda, Tabela.Custos$Demanda, Tabela.Custos$Estoque.Inicial)
# CRIANDO A COLUNA ESTOQUE FINAL
Tabela.Custos$Estoque.Final <- mapply(funcao_estoque.final, Tabela.Custos$Estoque.Inicial, Tabela.Custos$Venda)
# CRIANDO A COLUNA CUSTO DE FALTA
Tabela.Custos$Custo.Falta <- mapply(funcao_custo.falta, 
                                          Tabela.Custos$Demanda, Tabela.Custos$Estoque.Inicial)
# CRIANDO COLUNA CUSTO SOBRA 
Tabela.Custos$Custo.Sobra <- funcao_custo.sobra(Tabela.Custos$Estoque.Final)
# CRIANDO A COLUNA CUSTO TOTAL
Tabela.Custos$Custo.Total <- funcao_custo.total(Tabela.Custos$Custo.Falta, Tabela.Custos$Custo.Sobra)

In [15]:
Tabela.Custos

Semana,Sorteio,Demanda,Estoque.Inicial,Venda,Estoque.Final,Custo.Falta,Custo.Sobra,Custo.Total
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,956,255,233,233,0,3300,0,3300
2,622,237,255,237,18,0,7200,7200
3,421,231,237,231,6,0,2400,2400
4,153,219,231,219,12,0,4800,4800
5,169,219,219,219,0,0,0,0
6,472,231,219,219,0,1800,0,1800
7,60,213,231,213,18,0,7200,7200
8,697,237,213,213,0,3600,0,3600
9,712,237,237,237,0,0,0,0


In [16]:
Custo.Total <- sum(Tabela.Custos$Custo.Total)
cat("Custo Total:", Custo.Total,"u.m")

Custo Total: 55800 u.m

#### TESTANDO A SEGUNDA HIPÓTESE
2. COMPRAR UMA QUANTIDADE IGUAL À MÉDIA HISTÓRICA ANOTADA NO PERÍODO ANTERIOR DE 40 SEMANAS (MÉDIA = VALOR INTEIRO MAIS PRÓXIMO DA MÉDIA VERIFICADA).

In [17]:
cat("Média Ponderada =" ,media.ponderada, "\nComprar",media.ponderada,"toda semana")

Média Ponderada = 233 
Comprar 233 toda semana

In [18]:
# CRIANDO A TABELA DE CUSTO DA SEGUNDA HIPÓTESE COM BASE NA PRIMEIRA TABELA GERADA
Tabela.Custos2 = Tabela.Custos

In [19]:
# CRIANDO A COLUNA ESTOQUE INICIAL COM A MÉDIA PONDERADA
Tabela.Custos2$Estoque.Inicial[1:20] <- Estoque.Inicial

In [20]:
# CRIANDO A COLUNA VENDA
Tabela.Custos2$Venda <- mapply(funcao_venda, Tabela.Custos2$Demanda, Tabela.Custos2$Estoque.Inicial)
# CRIANDO A COLUNA ESTOQUE FINAL
Tabela.Custos2$Estoque.Final <- mapply(funcao_estoque.final, Tabela.Custos2$Estoque.Inicial, Tabela.Custos2$Venda)
# CRIANDO A COLUNA CUSTO DE FALTA
Tabela.Custos2$Custo.Falta <- mapply(funcao_custo.falta, 
                                          Tabela.Custos2$Demanda, Tabela.Custos2$Estoque.Inicial)
# CRIANDO COLUNA CUSTO SOBRA 
Tabela.Custos2$Custo.Sobra <- funcao_custo.sobra(Tabela.Custos2$Estoque.Final)
# CRIANDO A COLUNA CUSTO TOTAL
Tabela.Custos2$Custo.Total <- funcao_custo.total(Tabela.Custos2$Custo.Falta, Tabela.Custos2$Custo.Sobra)

In [21]:
Tabela.Custos2

Semana,Sorteio,Demanda,Estoque.Inicial,Venda,Estoque.Final,Custo.Falta,Custo.Sobra,Custo.Total
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,956,255,233,233,0,3300,0,3300
2,622,237,233,233,0,600,0,600
3,421,231,233,231,2,0,800,800
4,153,219,233,219,14,0,5600,5600
5,169,219,233,219,14,0,5600,5600
6,472,231,233,231,2,0,800,800
7,60,213,233,213,20,0,8000,8000
8,697,237,233,233,0,600,0,600
9,712,237,233,233,0,600,0,600


In [22]:
Custo.Total2 <- sum(Tabela.Custos2$Custo.Total)
cat("Custo Total:", Custo.Total2,"u.m")

Custo Total: 38000 u.m

O SEGUNDO QUADRO FOI, ATÉ AGORA, O MAIS FAVORÁVEL. OBESERVANDO A DISTRIBUIÇÃO DOS CUSTOS PARECE RAZOÁVEL PENSAR NA HIPÓTESE DE UMA COMPRA MENOR QUE A MÉDIA HISTÓRICA.

#### TESTANDO A TERCEIRA HIPÓTESE
3. O EXAME DOS RESULTADOS SUGERE O TESTE DE OUTRA HIPÓTESE.

In [23]:
cat("Compra de 230 dúzias por semana")

Compra de 230 dúzias por semana

In [24]:
# CRIANDO A TABELA DE CUSTO DA TERCEIRA HIPÓTESE COM BASE NA PRIMEIRA TABELA GERADA
Tabela.Custos3 = Tabela.Custos

In [25]:
# CRIANDO A COLUNA ESTOQUE INICIAL COM A QUANTIDADE DE 230 DÚZIAS
Tabela.Custos3$Estoque.Inicial[1:20] <- 230

In [26]:
# CRIANDO A COLUNA VENDA
Tabela.Custos3$Venda <- mapply(funcao_venda, Tabela.Custos3$Demanda, Tabela.Custos3$Estoque.Inicial)
# CRIANDO A COLUNA ESTOQUE FINAL
Tabela.Custos3$Estoque.Final <- mapply(funcao_estoque.final, Tabela.Custos3$Estoque.Inicial, Tabela.Custos3$Venda)
# CRIANDO A COLUNA CUSTO DE FALTA
Tabela.Custos3$Custo.Falta <- mapply(funcao_custo.falta, 
                                          Tabela.Custos3$Demanda, Tabela.Custos3$Estoque.Inicial)
# CRIANDO COLUNA CUSTO SOBRA 
Tabela.Custos3$Custo.Sobra <- funcao_custo.sobra(Tabela.Custos3$Estoque.Final)
# CRIANDO A COLUNA CUSTO TOTAL
Tabela.Custos3$Custo.Total <- funcao_custo.total(Tabela.Custos3$Custo.Falta, Tabela.Custos3$Custo.Sobra)

In [27]:
Tabela.Custos3

Semana,Sorteio,Demanda,Estoque.Inicial,Venda,Estoque.Final,Custo.Falta,Custo.Sobra,Custo.Total
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,956,255,230,230,0,3750,0,3750
2,622,237,230,230,0,1050,0,1050
3,421,231,230,230,0,150,0,150
4,153,219,230,219,11,0,4400,4400
5,169,219,230,219,11,0,4400,4400
6,472,231,230,230,0,150,0,150
7,60,213,230,213,17,0,6800,6800
8,697,237,230,230,0,1050,0,1050
9,712,237,230,230,0,1050,0,1050


In [28]:
Custo.Total3 <- sum(Tabela.Custos3$Custo.Total)
cat("Custo Total:", Custo.Total3,"u.m")

Custo Total: 34350 u.m

CONCLUSÃO: DAS TRÊS HIPÓTESES TESTADAS, A TERCEIRA PARECE A MAIS FAVORÁVEL.